### Code to extract data from naukri.com (June 2023)


Things to improve:
- things to add : by-pass the cookie and policy option.

In [1]:
# Author: Priti Gupta
# Date: June 8th, 2023
# Description: Scrapping data from Naukri to analyse salaries of data science positions in India
# GitHub:https://github.com/PritiG1/DS-SalaryPredictor


# import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import urllib.parse
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import csv
import os

def scraper_naukri(keyword_job):
    """
    Function to scrape job data from Naukri website based on a given job keyword.
    """
    # Encode the keyword for URL compatibility
    keyword_job_encoded = urllib.parse.quote(keyword_job, safe='-')
    
    # Construct the URL with the encoded keyword
    url = 'https://www.naukri.com/data-scientist-jobs-in-india-?k=' + keyword_job_encoded + '&l=india&nignbevent_src=jobsearchDeskGNB'

    # Specify the path to the Chrome WebDriver
    path = '/Users/pritigupta/Desktop/chromedriver_mac64/chromedriver'
    
    # Create a Chrome WebDriver instance
    driver = webdriver.Chrome(path)
    
    # Open the URL in the WebDriver
    driver.get(url)
    
    # Wait for the page to load (sleep for 15 seconds)
    time.sleep(15)

    # Check if the CSV file is empty to determine column names
    with open('datascientist_salary_naukri.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        file_empty = csvfile.tell() == 0
        if file_empty:
            column_names = ['Title', 'Company', 'Experience', 'Location', 'Salary']
        else:
            column_names = next(reader)

    # Open the CSV file in append mode to write job data
    with open('datascientist_salary_naukri.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write column names if the file is empty
        if file_empty:
            writer.writerow(column_names)

        # Iterate over job pages (up to 489 pages)
        for _ in range(1, 490):
            # Find all job elements on the page
            all_jobs = driver.find_elements(By.XPATH, '//section[@class="listContainer fleft"]')

            # Iterate over each job element
            for job in all_jobs:
                job_elements = job.find_elements(By.XPATH, './/article[@class="jobTuple"]')
                for element in job_elements:
                    try:
                        # Extract job details (title, company, experience, location, salary) and write to CSV file
                        writer.writerow([
                            element.find_element(By.XPATH, './/a[@class="title ellipsis"]').text,
                            element.find_element(By.XPATH, './/a[@class="subTitle ellipsis fleft"]').text,
                            element.find_element(By.XPATH, './/span[@class="ellipsis fleft expwdth"]').text,
                            element.find_element(By.XPATH, './/span[@class="ellipsis fleft locWdth"]').text,
                            element.find_element(By.XPATH, './/span[@class="ellipsis fleft "]').text
                        ])
                    except NoSuchElementException:
                        continue
        
            # Wait for the "Next" button to be clickable
            wait = WebDriverWait(driver, 15)
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH,'.//a[@class="fright fs14 btn-secondary br2"]')))
            
            # Click the "Next" button
            next_button.click()
            
            # Wait for the page to load (sleep for 15 seconds)
            time.sleep(15)

    # Quit the WebDriver
    driver.quit()

    # Return True to indicate successful execution
    return True

# Call the scraper function with the desired job keyword
data = scraper_naukri('Data Scientist')


/var/folders/_j/v313hjfd57bb0h0l_zx_wnt00000gn/T/ipykernel_4148/3182731978.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [5]:
df = pd.read_csv('datascientist_salary_naukri.csv')